In [1]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

In [2]:
# Create a SparkSession
spark = SparkSession.builder.appName("YDN EDA").getOrCreate()

In [4]:
df = spark.read.parquet("data/raw_data_*.parquet")
df.show()

+--------------------+-------------------+--------------------+--------+--------------------+------------------------------+
|                 url|               date|               title|subtitle|             content|estimated_reading_time_minutes|
+--------------------+-------------------+--------------------+--------+--------------------+------------------------------+
|https://yaledaily...|2000-10-09 00:00:00|Women's soccer fa...|    NULL|This game feature...|                           4.0|
|https://yaledaily...|2000-10-09 00:00:00|Fellowship season...|    NULL|A few days after ...|                           2.0|
|https://yaledaily...|2000-10-09 00:00:00|Dartmouth corners...|    NULL|Heading into Hano...|                           4.0|
|https://yaledaily...|2000-10-09 00:00:00|Yale panel debate...|    NULL|What is the defin...|                           2.0|
|https://yaledaily...|2000-10-09 00:00:00|Who's the real cr...|    NULL|To the Editor:\nI...|                           1.0|


In [5]:
df.count(), len(df.columns)

(79746, 6)

In [6]:
df.agg(
    F.mean("estimated_reading_time_minutes").alias("Average Estimated Reading Time"),
    F.max("estimated_reading_time_minutes").alias("Max Estimated Reading Time"),
).show()

+------------------------------+--------------------------+
|Average Estimated Reading Time|Max Estimated Reading Time|
+------------------------------+--------------------------+
|            3.6233225358630263|                      59.0|
+------------------------------+--------------------------+



In [7]:
max_reading_time = df.agg(F.max("estimated_reading_time_minutes")).collect()[0][0]

In [8]:
df.select("url").where(F.col("estimated_reading_time_minutes") == max_reading_time).collect()[0][0]

'https://yaledailynews.com/blog/2024/04/22/live-police-begin-arresting-pro-divestment-protesters-on-beinecke-plaza/'

In [12]:
df.withColumn("year", F.year("date")).groupBy("year").count().sort("year").show()

+----+-----+
|year|count|
+----+-----+
|2000|   20|
|2001| 2779|
|2002| 3510|
|2003| 3167|
|2004| 3089|
|2005| 3340|
|2006| 3270|
|2007| 3474|
|2008| 4316|
|2009| 4598|
|2010| 5109|
|2011| 5023|
|2012| 4347|
|2013| 3987|
|2014| 3677|
|2015| 3863|
|2016| 3887|
|2017| 3076|
|2018| 3058|
|2019| 2712|
+----+-----+
only showing top 20 rows



In [16]:
yearly_publications = df.withColumn("year", F.year("date")).groupBy("year").count().sort("year").toPandas()
yearly_publications.plot.line(x="year", y="count")

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

In [17]:
spark.stop()